In [2]:
import gym
from gym import spaces
from typing import Tuple, List, Dict
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt

import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset


In [3]:
from EVRoutePlanner import EVRoutePlanner
# from EVChargingEnv import EVChargingEnv

/Users/mattraghu/Documents/Projects/Hack-CEWIT-2024/EmissionsCalculator.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  power_plants = pd.read_csv("global_power_plant_database.csv", header=0)


In [4]:
with open("./APIKey.txt") as f:
    api_key = f.readline()

In [5]:
start = (40.783611, -74.698120)  # Chester, NJ
end = (42.886448, -78.878372) # Buffalo, NY

In [6]:
class EVRouteGymEnv(gym.Env):
    def __init__(self, ev_route_planner: EVRoutePlanner):
        super(EVRouteGymEnv, self).__init__()
        self.ev_route_planner = ev_route_planner
        self.action_space = spaces.Discrete(2)  # 0 = continue, 1 = charge
        self.observation_space = spaces.Box(low=0, high=float('inf'), shape=(2,), dtype=float)
        self.current_node_index = 0
        self.nodes = []
        self.remaining_range = ev_route_planner.vehicle_range

    def reset(self):
        self.nodes = self.ev_route_planner.calculate_route()
        self.current_node_index = 0
        self.remaining_range = self.ev_route_planner.vehicle_range
        return self._get_obs()

    def step(self, action):
        done = False
        reward = 0

        


        if action == 1:  # Charge
            if 'nearest_ev_station_distance' in self.nodes[self.current_node_index] and \
               self.remaining_range >= self.nodes[self.current_node_index]['nearest_ev_station_distance']:
                # Assuming charging restores the EV's range to its maximum capacity
                self.remaining_range = self.ev_route_planner.vehicle_range
                # Move to charging station if not at the current node's location
                self.remaining_range -= self.nodes[self.current_node_index].get('nearest_ev_station_distance', 0)
            else:
                # If charging is not possible due to range issues, heavily penalize the action
                reward = self._calculate_reward(action, False)
        elif action == 0:  # Continue
            if self.current_node_index < len(self.nodes) - 1:
                self.remaining_range -= self.nodes[self.current_node_index + 1]['dist']
                # self.current_node_index += 1
                reward = self._calculate_reward(action, True)
            else:
                done = True
                reward = self._calculate_reward(action, True)

        # Check if the EV can no longer move due to insufficient range, ending the episode
        if self.remaining_range < 0:
            done = True
            reward = self._calculate_reward(action, False)

        if not done:
            self.current_node_index += 1

        return self._get_obs(), reward, done, 'branched' in self.nodes[self.current_node_index-1]


    def _get_obs(self):
        # branched = 'branched' in self.nodes[self.current_node_index]
        if self.current_node_index < len(self.nodes):
            node = self.nodes[self.current_node_index]

            return [self.remaining_range, 'nearest_ev_station_distance' in node and node['nearest_ev_station_distance'] or 7777777]
        else:
            return [0, 0]

    def _can_charge(self):
        # Implement logic to check if the current node has a charging station and is within range
        current_node = self.nodes[self.current_node_index]
        return 'nearest_ev_station' in current_node and self.remaining_range >= current_node['nearest_ev_station_distance']

    def _calculate_reward(self, action, successful):
        # Reward logic considering the action's success and its implications
        if action == 1:  # Charge
            if successful:
                # Negative reward for charging to reflect time spent, but less penalty if it was necessary
                return -5  # Example penalty value, adjust based on needs
            else:
                # Heavy penalty if charging was attempted but is not possible
                return -100  # Example penalty value, adjust based on needs
        elif action == 0:  # Continue
            if successful:
                # Positive reward for successfully moving to the next node without unnecessary charging
                return 10  # Example reward value, adjust based on needs
            else:
                # Penalize running out of range
                return -100  # Example penalty value, adjust based on needs



In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

class EVDecisionNetwork(nn.Module):
    def __init__(self):
        super(EVDecisionNetwork, self).__init__()
        self.fc1 = nn.Linear(2, 64)  # 2 input features: remaining range and nearest station range
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # 2 output classes: 0 = don't branch, 1 = branch

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # Output raw scores for each class
        return x

# Create an instance of the network
ev_decision_network = EVDecisionNetwork()

# Example input: remaining range = 150 km, nearest station range = 50 km
example_input = torch.tensor([150, 50]).float()

# Forward pass through the network
output = ev_decision_network(example_input.unsqueeze(0))  # Add batch dimension

# Convert the output scores to probabilities
probabilities = torch.softmax(output, dim=1)


In [9]:
ev_route_planner = EVRoutePlanner(api_key, start, end, 200, 100)

In [13]:
training_data = []
def generate_training_data(env, episodes, steps_per_episode):

    for episode in range(episodes):
        state = env.reset()
        for step in range(steps_per_episode):
            action = random.choice([0, 1])  # Randomly choose to continue or charge
            next_state, reward, done, branched = env.step(action)

            # Record the state, action, and whether the action resulted in branching
            training_data.append((state, action, int(branched)))

            state = next_state
            if done:
                break

    return training_data

# Example usage
env = EVRouteGymEnv(ev_route_planner)
training_data = generate_training_data(env, episodes=5, steps_per_episode=10)



HERE
Processing node 1 of 18
Current distance: 33843.7495108047
Processing node 2 of 18
Current distance: 85978.27557038286
Processing node 3 of 18
Current distance: 74875.9732394834
Processing node 4 of 18
Current distance: 108178.2155518879
Processing node 5 of 18
Current distance: 141507.56550964335
Processing node 6 of 18
Current distance: 174725.9430950725
Processing node 7 of 18
Current distance: 190380.9914522095
Processing node 8 of 18
Current distance: 221837.67094505715
needs charge
setting to branch
Total Distance:  184638.9914522095
Added emissions:  74.6572220672783
HERE
Processing node 1 of 11
Current distance: 24461.900257752768
Processing node 2 of 11
Current distance: 72676.40112535238
Processing node 3 of 11
Current distance: 82187.36700729642
Processing node 4 of 11
Current distance: 115029.09185098344
Processing node 5 of 11
Current distance: 128324.04165311051
Processing node 6 of 11
Current distance: 159381.91888409675
Processing node 7 of 11
Current distance: 195

In [14]:
# Extract inputs and labels
inputs = torch.tensor([data[0] for data in training_data]).float()  # Convert to float for NN processing
labels = torch.tensor([data[1] for data in training_data]).long()  # Convert to long because these are categorical labels

# Create a TensorDataset and DataLoader
train_dataset = TensorDataset(inputs, labels)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Step 2: Define the Network, Loss Function, and Optimizer
# model = EVDecisionNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 3: Training Loop
num_epochs = 100  # Number of epochs to train for

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
    
    # Optionally print the loss every x epochs
    if epoch % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete.")


Epoch [1/100], Loss: 84954.4844
Epoch [2/100], Loss: 65204.1055
Epoch [3/100], Loss: 7887.4976
Epoch [4/100], Loss: 0.0000
Epoch [5/100], Loss: 451.5566
Epoch [6/100], Loss: 136.5991
Epoch [7/100], Loss: 39439.8438
Epoch [8/100], Loss: 15865.1875
Epoch [9/100], Loss: 21126.9355
Epoch [10/100], Loss: 0.0000
Epoch [11/100], Loss: 0.0000
Epoch [12/100], Loss: 100627.2266
Epoch [13/100], Loss: 717.9932
Epoch [14/100], Loss: 0.0000
Epoch [15/100], Loss: 0.0000
Epoch [16/100], Loss: 0.0000
Epoch [17/100], Loss: 425.7866
Epoch [18/100], Loss: 0.0000
Epoch [19/100], Loss: 362.3250
Epoch [20/100], Loss: 0.0000
Epoch [21/100], Loss: 59529.0938
Epoch [22/100], Loss: 0.0060
Epoch [23/100], Loss: 1283.6094
Epoch [24/100], Loss: 32339.6914
Epoch [25/100], Loss: 157.2122
Epoch [26/100], Loss: 0.0000
Epoch [27/100], Loss: 0.0000
Epoch [28/100], Loss: 192.0347
Epoch [29/100], Loss: 3261.3193
Epoch [30/100], Loss: 1702.3438
Epoch [31/100], Loss: 302.0398
Epoch [32/100], Loss: 40584.9102
Epoch [33/100], 

In [23]:
training_data

[([300000.0, 11318], 0, 0),
 ([274596.4739404218, 38049], 1, 0),
 ([261951.0, 2307], 1, 0),
 ([297693.0, 6457], 1, 0),
 ([293543.0, 13448], 1, 0),
 ([286552.0, 18630], 0, 0),
 ([258008.951642863, 5742], 1, 0),
 ([294258.0, 10395], 0, 0),
 ([270694.9440345842, 14109], 1, 0),
 ([285891.0, 13177], 0, 0),
 ([262987.6322883364, 8961], 1, 0),
 ([291039.0, 7777777], 0, 1),
 ([290589.0, 7777777], 1, 0),
 ([290589.0, 7777777], 0, 0),
 ([286929.0, 7777777], 0, 0),
 ([286918.0, 7777777], 1, 0),
 ([286918.0, 7777777], 0, 0),
 ([258440.86983344203, 5778], 1, 0),
 ([294222.0, 10457], 1, 0),
 ([289543.0, 20126], 1, 0),
 ([279874.0, 6733], 0, 0),
 ([253266.29147512728, 8956], 1, 0),
 ([291044.0, 22666], 0, 0),
 ([268518.2504891953, 11318], 0, 0),
 ([243114.72442961714, 38049], 1, 0),
 ([261951.0, 2307], 0, 0),
 ([232798.7576875955, 6457], 1, 0),
 ([293543.0, 7777777], 1, 1),
 ([293543.0, 7777777], 0, 0),
 ([283498.0, 7777777], 1, 0),
 ([283498.0, 7777777], 1, 0),
 ([283498.0, 7777777], 1, 0),
 ([28349